In [ ]:
# !pip3 install --quiet tensorflow==1.15
# !pip3 install --quiet "tensorflow_hub>=0.6.0"
# !pip3 install --quiet tensorflow_text==1.15

import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd
import numpy as np
import itertools
import datetime
import pickle

from collections import Counter
from sklearn.model_selection import train_test_split
from time import time

from gensim.models import KeyedVectors

from keras.models import load_model, Model, Sequential
from keras.layers import Bidirectional, Input, Embedding, Activation, Dense, Concatenate, Reshape, Dropout
from keras.layers.recurrent import LSTM
import keras.backend as K
from keras.callbacks import Callback,ModelCheckpoint, EarlyStopping
from keras.layers import Flatten, Lambda
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive/


In [ ]:
file = '/gdrive/My Drive/NLA_project/new/data_unstemmed.csv' 
vocab_path = '/gdrive/My Drive/NLA_project/new/vocabulary_full'
model_path = '/gdrive/My Drive/NLA_project/new/Siamese_CNN_univ_man.h5'
# data = pd.read_csv(file, names=["Ref_paper", "Cit_paper", "Cit_Text", "Ref_text", "label"])

data = pd.read_csv(file)

In [ ]:
data = data[['Cit_Text', 'Ref_text', 'label']]
print(len(data[data['label']==1]),len(data[data['label']==0]))
data.head()

33753 29989


,Cit_Text,Ref_text,label
0,finally use pos tagger trained tweets gimpel e...,contributions follows developed features twitt...,1
1,finally use pos tagger trained tweets gimpel e...,next obtained random sample mostly american en...,1
2,finally use pos tagger trained tweets gimpel e...,figure 1 show examples certain features help s...,0
3,finally use pos tagger trained tweets gimpel e...,one fundamental parts linguistic pipeline part...,0
4,however pos taggers twitter available limited ...,pos taggers trained treebanks newswire domain ...,1


In [ ]:
#### CONSIDER REMOVING SENTENCE LENGTHS ABOVE 100
lower_lim = 3
upper_lim = 40


a, b, c = (data.values)[0]
print(str(a), str(b), str(c))

lis = list()
for a,b,c in data.values:
    a = str(a).split(' ')
    b = str(b).split(' ')
    
    if len(a) < lower_lim or len(b) < lower_lim:
        continue
    if len(a) > upper_lim:
        a = a[:upper_lim]
    if len(b) > upper_lim:
        b = b[:upper_lim]
        
    lis.append((' '.join(a),' '.join(b),c))

print(len(lis),len(data))
data = pd.DataFrame(lis , columns = ['Cit_Text', 'Ref_text', 'label'])
print(len(data[data['label']==1]),len(data[data['label']==0]))
data.head()

finally use pos tagger trained tweets gimpel et al 2011 perform pos tagging tweets data apart previously recognised named entities words tagged nouns verbs adjectives kept contributions follows developed features twitter pos tagging conducted experiments evaluate provide annotated corpus trained pos tagger research community 1
59654 63742
29917 29737


,Cit_Text,Ref_text,label
0,finally use pos tagger trained tweets gimpel e...,contributions follows developed features twitt...,1
1,finally use pos tagger trained tweets gimpel e...,next obtained random sample mostly american en...,1
2,finally use pos tagger trained tweets gimpel e...,figure 1 show examples certain features help s...,0
3,finally use pos tagger trained tweets gimpel e...,one fundamental parts linguistic pipeline part...,0
4,however pos taggers twitter available limited ...,pos taggers trained treebanks newswire domain ...,1


In [ ]:
### CROSS VALIDATION

# X1 = data['Cit_Text'].values
# X2 = data['Ref_text'].values
# y=data['label'].values

# skf = StratifiedKFold(n_splits=2)
# skf.get_n_splits(X, y)

# print(skf)

# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

In [ ]:
max_len = 41
import matplotlib.pyplot as plt
# %matplotlib qt
def load_data_and_labels():

    cit_text = data['Cit_Text'].values
    ref_text = data['Ref_text'].values

    cit_text = [str(temp).split(' ') for temp in cit_text]
    ref_text = [str(temp).split(' ') for temp in ref_text]
    
    x_text = [zipped for zipped in zip(cit_text, ref_text)]
    return x_text, data['label'].values

def pad_sentences(sentences, padding_word="<PAD/>"):
    
    global max_len
    max_len = max(max(len(x[0]), len(x[1])) for x in sentences)
    min_len =  min(min(len(x[0]), len(x[1])) for x in sentences)
    max_len += 1

    padded_sentences = list()
    for a,b in sentences:
        a += [padding_word] * (max_len - len(a))
        b += [padding_word] * (max_len - len(b))
        padded_sentences.append((a, b))
    return padded_sentences

def build_vocab(sentences):
    
    cit = list()
    ref = list()
    for a, b in sentences:
        cit.append(a)
        ref.append(b)
    word_counts = Counter(itertools.chain(*cit))
    dict.update(word_counts, Counter(itertools.chain(*ref)))
    X_plot = list()
    Y_plot = list()
    # for a,b in word_counts.most_common(1000):
    #   if b>=100000:
    #     continue
    #   X_plot.append(a)
    #   Y_plot.append(word_counts[a])
      # print("Yes")
    # plt.plot(X_plot, Y_plot)
    # plt.show()
    till = 0 
    for a,b in word_counts.most_common():
      if b<=5:
        break
      till += 1
      # print(a,b)
    print(len(word_counts.most_common(till)))
    vocabulary_inv = [x[0] for x in word_counts.most_common(till)]
    vocabulary_inv.append('<UNK>')
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

def build_input_data(sentences, labels, vocabulary):
    
    x1 = []
    for sentence in sentences:
        temp = []
        for word in sentence[0]:
            if str(word) in vocabulary:
                temp.append(vocabulary[str(word)])
            else:
                temp.append(vocabulary['<UNK>'])
        x1.append(temp)
    x2 = []
    for sentence in sentences:
        temp = []
        for word in sentence[1]:
            if str(word) in vocabulary:
                temp.append(vocabulary[str(word)])
            else:
                temp.append(vocabulary['<UNK>'])
        x2.append(temp)
    y = np.array(labels)
    return [np.array(x1), np.array(x2), y]

def save_vocab(vocabulary):
    dbfile = open(vocab_path, 'wb')
    pickle.dump(vocabulary, dbfile)                      
    dbfile.close()


def load_data():

    sentences, labels = load_data_and_labels()
    
    sentences = pad_sentences(sentences)
    print(len(sentences), len(sentences[0][0]))
    
    vocabulary, vocabulary_inv = build_vocab(sentences)
    save_vocab(vocabulary)
    
    X1, X2, y = build_input_data(sentences, labels, vocabulary)
    print(X1[:2], X2[:2], labels[:2])
    return [X1, X2, y, vocabulary, vocabulary_inv]

X1, X2, y, vocabulary, vocabulary_inv = load_data()
# load_data()
print(len(list(vocabulary.keys())))

59654 41
10310
[[  369    14   151   201   156  2694  5834     2     1   765   420   151
    131  2694     9  3104   494 10310   344   300     7   460   292   233
    669  2742     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [  369    14   151   201   156  2694  5834     2     1   765   420   151
    131  2694     9  3104   494 10310   344   300     7   460   292   233
    669  2742     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]] [[ 2609   363   333    23  1644   151   131  1624    86   373   346   252
     18   156   151   201   223  1471     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [  340   244   427   449  1437  2336 10310  2694  6033  2569   365   199
   4089     6  1644  5412  5588     2     1  4328    17  9721     6  9078
   1750   151   201  2123     2     1    43   105  1069   805   228    

In [ ]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")

# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

tf.logging.set_verbosity(tf.logging.ERROR)

session = tf.Session()
session.run([tf.global_variables_initializer(), tf.tables_initializer()])
message_embeddings = session.run(embed(messages))

for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  print("Message: {}".format(messages[i]))
  print("Embedding size: {}".format(len(message_embedding)))
  message_embedding_snippet = ", ".join(
      (str(x) for x in message_embedding[:3]))
  print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: Elephant
Embedding size: 512
Embedding: [0.044984716922044754, -0.05743396654725075, 0.002211471553891897, ...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding: [0.05568017438054085, -0.009607899002730846, 0.006246284581720829, ...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [0.038749415427446365, 0.0765201598405838, -0.0007946128025650978, ...]



In [ ]:

# # EMBEDDING_FILE = '..../GoogleNews-vectors-negative300.bin.gz'
# # word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
# tf.logging.set_verbosity(tf.logging.ERROR)

# session = tf.Session()
# session.run([tf.global_variables_initializer(), tf.tables_initializer()])
# embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
# print(np.array(vocabulary.items()).shape)
embedding_dim = 512
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)
embeddings[0] = 0  # So that the padding will be ignored

# embeds = session.run(embed(list(vocabulary.keys())))
# print(embeds.shape)
# for i, word in enumerate(list(vocabulary.keys())):
#   embeddings[vocabulary[word]] = embeds[i]

for word, index in vocabulary.items():
#     # if word in word2vec.vocab:
#     # embeddings[index] = np.array(embeds[index])
#     # else:
    embeddings[index] = np.random.uniform(-0.25,0.25,embedding_dim)
# del word2vec
print(embeddings.shape)
# session.close()



(10312, 512)


In [ ]:
import pickle
embed_f = '/gdrive/My Drive/NLA_project/embeddings_univ'
e_f = open(embed_f, 'wb')
pickle.dump(embeddings, e_f)
e_f.close()

In [ ]:
X = list()
for a, b in zip(X1, X2):
    X.append((a, b))
X_train,X_test, Y_train, Y_test =  train_test_split(X, y,test_size = 0.20,random_state= 4,shuffle=False)

training = []
for a,b in zip(X_train, Y_train):
    training.append((a,b))
np.random.shuffle(training)

X_train1 = list()
X_train2 = list()
Y_train = list()

X_test1 = list()
X_test2 = list()

for a, b in training:
    X_train1.append(a[0])
    X_train2.append(a[1])
    Y_train.append(b)
    
for a, b in X_test:
    X_test1.append(a)
    X_test2.append(b)
    
X_train1 = np.array(X_train1)
X_train2 = np.array(X_train2)
X_test1 = np.array(X_test1)
X_test2 = np.array(X_test2)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
print(len(X_train1), len(X_train2), len(Y_train))

47723 47723 47723


In [ ]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision  

In [ ]:
n_epoch = 10

left_input = Input(shape=(max_len,))
right_input = Input(shape=(max_len,))

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_len, trainable=True)

encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

encoder_lstm = Bidirectional (LSTM (300, return_sequences=False, dropout=0.50),merge_mode='concat')
first_out = encoder_lstm (encoded_left)
second_out = encoder_lstm (encoded_right)

def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([first_out, second_out])
merged = Concatenate(axis=1)([first_out, second_out])

output_layer1 = Dense(128, activation='relu')(merged)
output_layer2 = Dense(32, activation='relu')(output_layer1)
output_layer3 = Dense(1, activation='sigmoid')(output_layer2)

# model = Model(inputs=[left_input, right_input], outputs=malstm_distance)
model = Model(inputs=[left_input, right_input], outputs=output_layer3)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy', get_f1, recall, precision])
model.summary()

early_stopping = EarlyStopping(monitor="val_get_f1", mode='max',patience=3, verbose=True)

training_start_time = time()

model.fit([X_train1, X_train2], Y_train, epochs=n_epoch, validation_data=([X_test1, X_test2],Y_test),callbacks = [early_stopping, ModelCheckpoint(filepath=model_path, save_best_only=True)])

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

# saved_model = load_model(mode_path, custom_objects={"get_f1":get_f1, "precision":precision, "recall":recall})
# model.compile('adam', 'binary_crossentropy', metrics=['accuracy', get_f1, recall, precision])
# result = model.predict([X_test1, X_test2])
# print(result)
# model.evaluate([X_test1, X_test2], Y_test)


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 41)           0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 41)           0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 41, 512)      5279744     input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 600)          1951200     embedding_7[0][0]          

In [ ]:
n_epoch = 10
kernel_size = 5
filters = 128
pool_size = 4

lstm_output_size = 128

model_path = '/gdrive/My Drive/NLA_project/new/Siamese_LSTM_univ.h5'

left_input = Input(shape=(max_len,))
right_input = Input(shape=(max_len,))

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_len, trainable=True)

encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

encoded_left = Dropout(0.25) (encoded_left)
encoded_right = Dropout(0.25) (encoded_right)

cnn_share = Conv1D(filters,kernel_size,padding='valid',activation='relu',strides=1)

encoded_left1 = cnn_share (encoded_left)
encoded_right1 = cnn_share (encoded_right)

first_out1 = Flatten() (MaxPooling1D(pool_size=pool_size) (encoded_left1))
second_out1 = Flatten() (MaxPooling1D(pool_size=pool_size) (encoded_right1))

merged1 = Concatenate(axis=1)([first_out1, second_out1])


cnn_share2 = Conv1D(filters,3,padding='valid',activation='relu',strides=1)

encoded_left2 = cnn_share2 (encoded_left)
encoded_right2 = cnn_share2 (encoded_right)

first_out2 = Flatten() (MaxPooling1D(pool_size=pool_size) (encoded_left2))
second_out2 = Flatten() (MaxPooling1D(pool_size=pool_size) (encoded_right2))

merged2 = Concatenate(axis=1)([first_out2, second_out2])

cnn_share3 = Conv1D(filters,4,padding='valid',activation='relu',strides=1)

encoded_left3 = cnn_share3 (encoded_left)
encoded_right3 = cnn_share3 (encoded_right)

first_out3 = Flatten() (MaxPooling1D(pool_size=pool_size) (encoded_left3))
second_out3 = Flatten() (MaxPooling1D(pool_size=pool_size) (encoded_right3))

merged3 = Concatenate(axis=1)([first_out3, second_out3])

cnn_share4 = Conv1D(filters,6,padding='valid',activation='relu',strides=1)

encoded_left4 = cnn_share4 (encoded_left)
encoded_right4 = cnn_share4 (encoded_right)

first_out4 = Flatten() (MaxPooling1D(pool_size=pool_size) (encoded_left4))
second_out4 = Flatten() (MaxPooling1D(pool_size=pool_size) (encoded_right4))

merged4 = Concatenate(axis=1)([first_out4, second_out4])


merged = Concatenate(axis=1) ([merged1, merged2, merged3, merged4])
# merged = Concatenate(axis=1) ([merged1, merged2, merged3])
output_layer1 = Dense(128, activation='relu')(merged)
output_layer2 = Dense(32, activation='relu')(output_layer1)
output_layer3 = Dense(1, activation='sigmoid')(output_layer2)

model = Model(inputs=[left_input, right_input], outputs=output_layer3)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy', get_f1, recall, precision])
model.summary()

early_stopping = EarlyStopping(monitor="val_get_f1", mode='max', patience=2, verbose=True)
training_start_time = time()

model.fit([X_train1, X_train2], Y_train, epochs=n_epoch, validation_data=([X_test1, X_test2], Y_test),callbacks = [early_stopping, ModelCheckpoint(filepath=model_path, save_best_only=True)])

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 41)           0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 41)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 41, 512)      5279744     input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 41, 512)      0           embedding_6[0][0]          

In [ ]:
# result = model.predict([X_test1, X_test2])
# print(result)
model.evaluate([X_test1, X_test2], Y_test)

11931/11931 [==============================] - 30s 2ms/step


[0.8631682172224054,
 0.6238370537757874,
 0.5868266224861145,
 0.5537066459655762,
 0.6508835554122925]